In [2]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys
import numpy as np 
import pandas as pd
import nibabel as nb
import ast
import pydicom

import Diffusion_motion_field.functions_collection as ff

main_path = '/mnt/camca_NAS/4DCT/pictures'

### patient list

In [18]:
patient_list = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_MVF_diffusion_train_test.xlsx')
ef_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx')
correspondence = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_name_correspondance.xlsx')

# patient_query_list = patient_list[patient_list['batch'] == 5]
# print('query list length:', len(patient_query_list))

### make demographics

In [20]:
results = []
for i in range(0,patient_list.shape[0]):
    row = patient_list.iloc[i]
    patient_class = row['patient_class']
    patient_id = row['patient_id']
    print(patient_class, patient_id)

    # batch 
    batch = row['batch']

    # image info
    pixel_dim = ast.literal_eval(row['pix_dim'])
    pixel_dim_xy = np.round(pixel_dim[0],4)
    pixel_dim_z = np.round(pixel_dim[2],4)

    shape = ast.literal_eval(row['shape'])
    shape_xy = shape[0]
    shape_z = shape[2]

    # EF
    ef_row = ef_info[ef_info['patient_id'] == patient_id]
    ef = np.round(ef_row['EF_sampled_in_10tf_by_mvf'].values[0],4)

    # DICOM info
    correspondence_row = correspondence[correspondence['patient_id'] == patient_id]
    case_dicom_folder = correspondence_row['case_dicom_folder'].values[0]
    
    dicom_files = ff.find_all_target_files(['*.dcm'],case_dicom_folder)
    if len(dicom_files) == 0:
        dicom_files = ff.find_all_target_files(['*/*.dcm'],case_dicom_folder)
    dicom_file = dicom_files[0]
    dcm = pydicom.dcmread(dicom_files[0])

    manufacturer= dcm.Manufacturer if hasattr(dcm, 'Manufacturer') else ''
    model = dcm.ManufacturerModelName if hasattr(dcm,'ManufacturerModelName') else ''
    studydate = dcm.StudyDate if hasattr(dcm,'StudyDate') else ''
    age = dcm.PatientAge if hasattr(dcm,'PatientAge') else ''
    age = int(age[1:3])
    sex = dcm.PatientSex if hasattr(dcm,'PatientSex') else ''
    protocol = dcm.ProtocolName if hasattr(dcm,'ProtocolName') else ''

    results.append([patient_class, patient_id, batch, pixel_dim_xy, pixel_dim_z, shape_xy, shape_z, ef,manufacturer, model, studydate, age, sex,protocol])
    df = pd.DataFrame(results, columns = ['patient_class','patient_id', 'batch', 'pix_dim_xy','pix_dim_z','shape_xy','shape_z','EF',
                                          'manufacturer', 'model','studydate','age','sex','protocol'])
    df.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/scan_info.xlsx', index=False)


Pull_2 case_66
Pull_4 case_185
Pull_3 case_103
Pull_4 case_198
Pull_4 case_152
Pull_4 case_206
Pull_1 case_24
Pull_1 case_29
Pull_1 case_25
Pull_1 case_15
Pull_3 case_105
Pull_4 case_151
Pull_3 case_90
Pull_3 case_89
Pull_4 case_208
Pull_2 case_48
Pull_4 case_200
Pull_3 case_93
Pull_4 case_150
Pull_3 case_92
Pull_4 case_213
Pull_4 case_183
Pull_4 case_158
Pull_3 case_129
Pull_3 case_91
Pull_4 case_154
Pull_1 case_38
Pull_4 case_215
Pull_2 case_83
Pull_4 case_196
Pull_4 case_153
Pull_4 case_224
Pull_1 case_32
Pull_1 case_28
Pull_2 case_80
Pull_3 case_120
Pull_4 case_231
Pull_1 case_9
Pull_4 case_209
Pull_4 case_197
Pull_4 case_223
Pull_3 case_111
Pull_4 case_147
Pull_4 case_220
Pull_2 case_55
Pull_4 case_184
Pull_2 case_63
Pull_4 case_247
Pull_3 case_133
Pull_2 case_77
Pull_3 case_101
Pull_4 case_172
Pull_2 case_47
Pull_4 case_232
Pull_2 case_78
Pull_2 case_73
Pull_4 case_221
Pull_2 case_59
Pull_2 case_49
Pull_4 case_210
Pull_1 case_20
Pull_2 case_62
Pull_4 case_226
Pull_4 case_266
Pull

## statistics

In [32]:
import Diffusion_motion_field.Build_lists.Build_list as Build_list
a = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/scan_info.xlsx'))
print('a.shape', a.shape)
data_sheet = os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_MVF_diffusion_train_test.xlsx')
b = Build_list.Build(data_sheet)
patient_class_list_train, patient_id_list_train,_ = b.__build__(batch_list = [0,1,2,3,4])
patient_class_list_test, patient_id_list_test,_ = b.__build__(batch_list = [5])

train_data = a[a['patient_id'].isin(patient_id_list_train)]
test_data = a[a['patient_id'].isin(patient_id_list_test)]

from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu

a.shape (208, 14)


In [33]:
total_age = a['age']
print('mean and std age:', np.mean(total_age), np.std(total_age))
train_age = train_data['age']
print('mean and std age train:', np.mean(train_age), np.std(train_age))
test_age = test_data['age']
print('mean and std age test:', np.mean(test_age), np.std(test_age))

t_stat, p_value = ttest_ind(train_age, test_age)
print('T-test between train and test age:', t_stat, p_value)

mean and std age: 59.12019230769231 15.156236846514147
mean and std age train: 60.035714285714285 14.922774563742228
mean and std age test: 55.275 15.52093344486729
T-test between train and test age: 1.7905614022897576 0.07483209480975206


In [34]:
total_male_percent = a.loc[a['sex'] == 'M'].shape[0] / a.shape[0]
train_male_percent = train_data.loc[train_data['sex'] == 'M'].shape[0] / train_data.shape[0]
test_male_percent = test_data.loc[test_data['sex'] == 'M'].shape[0] / test_data.shape[0]
print('percent list: ',total_male_percent, train_male_percent, test_male_percent)

table = [[train_data.loc[train_data['sex'] == 'M'].shape[0], train_data.shape[0] - train_data.loc[train_data['sex'] == 'M'].shape[0]],
         [test_data.loc[test_data['sex'] == 'M'].shape[0], test_data.shape[0] - test_data.loc[test_data['sex'] == 'M'].shape[0]]]

chi2, p, dof, expected = chi2_contingency(table)

print("Chi-square statistic:", chi2)
print("P-value:", p)

percent list:  0.4423076923076923 0.42857142857142855 0.5
Chi-square statistic: 0.4100378382237448
P-value: 0.5219503007945621


In [38]:
total_EF = a['EF']
train_EF = train_data['EF']
test_EF = test_data['EF']
print('medians: ', np.median(total_EF), np.median(train_EF), np.median(test_EF))
stat, p = mannwhitneyu(train_EF, test_EF, alternative='two-sided')
print('T-test between train and test EF:', stat, p)

# <0.4
total_count = np.sum(a['EF'] < 0.40) 
train_count= np.sum(train_data['EF'] <0.40) 
test_count = np.sum(test_data['EF'] < 0.40) 
print('count list: ', total_count, train_count, test_count)
# table = [[np.sum(train_data['EF'] < 0.40), train_data.shape[0] - np.sum(train_data['EF'] < 0.40)],
#             [np.sum(test_data['EF'] < 0.40), test_data.shape[0] - np.sum(test_data['EF'] < 0.40)]]
# chi2, p, dof, expected = chi2_contingency(table)
# print('p value for <40% EF:', p)

# 0.4~0.5
total_count = np.sum((a['EF'] >= 0.40) & (a['EF'] < 0.50))
train_count = np.sum((train_data['EF'] >= 0.40) & (train_data['EF'] < 0.50))
test_count = np.sum((test_data['EF'] >= 0.40) & (test_data['EF'] < 0.50))
print('count list: ', total_count, train_count, test_count)

# >0.5
total_count = np.sum(a['EF'] >= 0.50)
train_count = np.sum(train_data['EF'] >= 0.50)
test_count = np.sum(test_data['EF'] >= 0.50)
print('count list: ', total_count, train_count, test_count)




medians:  0.58195 0.5911500000000001 0.54075
T-test between train and test EF: 4270.0 0.007848888000378652
count list:  27 15 12
count list:  28 23 5
count list:  153 130 23
